In [2]:
import os

import b3d
import b3d.bayes3d as bayes3d
import fire
import genjax
import jax
import jax.numpy as jnp
import numpy as np
import h5py
import rerun as rr
from b3d import Pose
from genjax import Pytree
from tqdm import tqdm
genjax.pretty()

ModuleNotFoundError: No module named 'h5py'

In [1]:
filename = os.path.join(
    b3d.get_root_path(),
    "assets/physion/hdf5s/pilot_dominoes_0mid_d3chairs_o1plants_tdwroom_0012.hdf5",
)
frame = 0000
with h5py.File(filename, "r") as f:
    depth_data = f[f'frames/{frame}/images/_depth_cam0']  # Access _img data for each frame _depth_cam0
    depth = np.array(depth_data)
    depth_reshape = depth.reshape(512, 512)

NameError: name 'os' is not defined

In [ ]:
video_input = b3d.io.VideoInput(
    rgb=jnp.array(rgb)[None, ...],
    xyz=b3d.xyz_from_depth(jnp.array(depth), fx, fy, cx, cy)[None, ...],
    camera_intrinsics_rgb=jnp.array(
        [image_width, image_height, fx, fy, cx, cy, 0.01, 10.0]
    ),
    camera_intrinsics_depth=jnp.array(
        [image_width, image_height, fx, fy, cx, cy, 0.01, 10.0]
    ),
    camera_positions=jnp.zeros((1, 3)),
    camera_quaternions=jnp.zeros((1, 4)),
)

In [2]:
rr.init("demo")
rr.connect("127.0.0.1:8812")

In [3]:
path = os.path.join(
    b3d.get_root_path(),
    "assets/shared_data_bucket/input_data/shout_on_desk.r3d.video_input.npz",
)
video_input = b3d.io.VideoInput.load(path)

# Get intrinsics
image_width, image_height, fx, fy, cx, cy, near, far = np.array(
    video_input.camera_intrinsics_depth
)
image_width, image_height = int(image_width), int(image_height)
fx, fy, cx, cy, near, far = (
    float(fx),
    float(fy),
    float(cx),
    float(cy),
    float(near),
    float(far),
)

# Get RGBS and Depth
rgbs = video_input.rgb[::4] / 255.0
xyzs = video_input.xyz[::4]

In [4]:
image_width, image_height

(192, 256)

In [5]:
fx, fy, cx, cy, near, far

(181.88333129882812,
 181.88333129882812,
 96.39157104492188,
 128.85830688476562,
 0.009999999776482582,
 100.0)

In [6]:
rgbs.shape

(186, 960, 720, 3)

In [7]:
xyzs.shape

(186, 256, 192, 3)

In [8]:
rgbs_resized = jnp.clip(
    jax.vmap(jax.image.resize, in_axes=(0, None, None))(
        rgbs, (video_input.xyz.shape[1], video_input.xyz.shape[2], 3), "linear"
    ),
    0.0,
    1.0,
)
rgbs_resized.shape

(186, 256, 192, 3)

In [9]:
# Make empty library
object_library = bayes3d.MeshLibrary.make_empty_library()

# Take point cloud at frame 0
point_cloud = jax.image.resize(
    xyzs[0], (xyzs[0].shape[0] // 3, xyzs[0].shape[1] // 3, 3), "linear"
).reshape(-1, 3)
colors = jax.image.resize(
    rgbs_resized[0], (xyzs[0].shape[0] // 3, xyzs[0].shape[1] // 3, 3), "linear"
).reshape(-1, 3)

# `make_mesh_from_point_cloud_and_resolution` takes a 3D positions, colors, and sizes of the boxes that we want
# to place at each position and create a mesh
vertices, faces, vertex_colors, _face_colors = (
    b3d.make_mesh_from_point_cloud_and_resolution(
        point_cloud,
        colors,
        point_cloud[:, 2]
        / fx
        * 3.0,  # This is scaling the size of the box to correspond to the effective size of the pixel in 3D. It really should be multiplied by 2.
        # and the 6 makes it larger
    )
)
object_library.add_object(vertices, faces, vertex_colors)

In [10]:
vertices.shape

(130560, 3)

In [11]:
vertex_colors.shape

(130560, 3)

In [13]:
vertex_colors

<jax.Array float32(130560, 3) ≈0.58 ±0.23 [≥0.023, ≤0.96] nonzero:391_680
  <Arrayviz rendering>
>

In [14]:
object_library

In [15]:
object_library.vertices

<jax.Array float32(130560, 3) ≈0.15 ±0.32 [≥-0.49, ≤0.86] nonzero:391_680
  <Arrayviz rendering>
>

In [18]:
object_library.faces

<jax.Array int32(65280, 3) [≥0, ≤130_559] zero:2 nonzero:195_838
  <Arrayviz rendering>
>

In [19]:
object_library.ranges

<Array([[    0, 65280]], dtype=int32)
  <Arrayviz rendering>
>

In [20]:
object_library.attributes

<jax.Array float32(130560, 3) ≈0.58 ±0.23 [≥0.023, ≤0.96] nonzero:391_680
  <Arrayviz rendering>
>

In [21]:
object_library.vertex_index_to_object

<jax.Array int32(130560,) [≥0, ≤0] zero:130_560
  <Arrayviz rendering>
>

In [22]:
object_library.objects

[(<jax.Array float32(130560, 3) ≈0.15 ±0.32 [≥-0.49, ≤0.86] nonzero:391_680
    <Arrayviz rendering>
  >,
  <jax.Array int32(65280, 3) [≥0, ≤130_559] zero:2 nonzero:195_838
    <Arrayviz rendering>
  >,
  <jax.Array float32(130560, 3) ≈0.58 ±0.23 [≥0.023, ≤0.96] nonzero:391_680
    <Arrayviz rendering>
  >)]

In [ ]:
# Gridding on translation only.
translation_deltas = Pose.concatenate_poses(
    [
        jax.vmap(lambda p: Pose.from_translation(p))(
            jnp.stack(
                jnp.meshgrid(
                    jnp.linspace(-0.01, 0.01, 11),
                    jnp.linspace(-0.01, 0.01, 11),
                    jnp.linspace(-0.01, 0.01, 11),
                ),
                axis=-1,
            ).reshape(-1, 3)
        ),
        Pose.identity()[None, ...],
    ]
)
# Sample orientations from a VMF to define a "grid" over orientations.
rotation_deltas = Pose.concatenate_poses(
    [
        jax.vmap(Pose.sample_gaussian_vmf_pose, in_axes=(0, None, None, None))(
            jax.random.split(jax.random.PRNGKey(0), 11 * 11 * 11),
            Pose.identity(),
            0.00001,
            1000.0,
        ),
        Pose.identity()[None, ...],
    ]
)
all_deltas = Pose.stack_poses([translation_deltas, rotation_deltas])


In [ ]:
all_deltas

In [ ]:
all_deltas[1]

In [ ]:
# Defines the enumeration schedule.
key = jax.random.PRNGKey(0)
renderer = b3d.Renderer(image_width, image_height, fx, fy, cx, cy, near, far)
model = bayes3d.model_multiobject_gl_factory(renderer)
importance_jit = jax.jit(model.importance)

# Arguments of the generative model.
# These control the inlier / outlier decision boundary for color error and depth error.
color_error, depth_error = (60.0, 0.01)
inlier_score, outlier_prob = (5.0, 0.00001)
color_multiplier, depth_multiplier = (10000.0, 500.0)
model_args = bayes3d.ModelArgs(
    color_error,
    depth_error,
    inlier_score,
    outlier_prob,
    color_multiplier,
    depth_multiplier,
)

In [ ]:
# Initial trace for timestep 0
START_T = 0
trace, _ = importance_jit(
    jax.random.PRNGKey(0),
    genjax.ChoiceMap.d(
        dict(
            [
                ("camera_pose", Pose.identity()),
                ("object_pose_0", Pose.identity()),
                ("object_0", 0),
                (
                    "observed_rgb_depth",
                    (rgbs_resized[START_T], xyzs[START_T, ..., 2]),
                ),
            ]
        )
    ),
    (jnp.arange(1), model_args, object_library),
)

In [ ]:
trace

In [ ]:
trace.get_retval()

In [ ]:
rgbs_resized[START_T]

In [ ]:
xyzs[START_T, ..., 2]

In [ ]:
# Visualize trace
bayes3d.rerun_visualize_trace_t(trace, 0)


In [ ]:
ACQUISITION_T = 90
for T_observed_image in tqdm(range(ACQUISITION_T)):
    # Constrain on new RGB and Depth data.
    trace = b3d.update_choices(
        trace,
        Pytree.const(["observed_rgb_depth"]),
        (rgbs_resized[T_observed_image], xyzs[T_observed_image, ..., 2]),
    )
    trace, key = bayes3d.enumerate_and_select_best_move(
        trace, Pytree.const(("camera_pose",)), key, all_deltas
    )
    bayes3d.rerun_visualize_trace_t(trace, T_observed_image)


In [ ]:
# Outliers are AND of the RGB and Depth outlier masks
_inliers, _color_inliers, _depth_inliers, outliers, _undecided, _valid_data_mask = (
    bayes3d.get_rgb_depth_inliers_from_trace(trace)
)
outlier_mask = outliers
rr.log("outliers", rr.Image(jnp.tile((outlier_mask * 1.0)[..., None], (1, 1, 3))))

# Get the point cloud corresponding to the outliers
rgb, depth = trace.get_choices()["observed_rgb_depth"]
point_cloud = b3d.xyz_from_depth(depth, fx, fy, cx, cy)[outlier_mask]
point_cloud_colors = rgb[outlier_mask]

# Segment the outlier cloud.
assignment = b3d.segment_point_cloud(point_cloud)

# Only keep the largers cluster in the outlier cloud.
point_cloud = point_cloud.reshape(-1, 3)[assignment == 0]
point_cloud_colors = point_cloud_colors.reshape(-1, 3)[assignment == 0]

# Create new mesh.
vertices, faces, vertex_colors, _face_colors = (
    b3d.make_mesh_from_point_cloud_and_resolution(
        point_cloud, point_cloud_colors, point_cloud[:, 2] / fx * 2.0
    )
)
object_pose = Pose.from_translation(vertices.mean(0))
vertices = object_pose.inverse().apply(vertices)
object_library.add_object(vertices, faces, vertex_colors)

In [ ]:
object_library

In [ ]:
object_library.ranges

In [ ]:
len(object_library.ranges)

In [ ]:
single_object_trace = trace
trace = single_object_trace

trace, _ = importance_jit(
    jax.random.PRNGKey(0),
    genjax.ChoiceMap.d(
        dict(
            [
                ("camera_pose", trace.get_choices()["camera_pose"]),
                ("object_pose_0", trace.get_choices()["object_pose_0"]),
                ("object_pose_1", trace.get_choices()["camera_pose"] @ object_pose),
                ("object_0", 0),
                ("object_1", 1),
                (
                    "observed_rgb_depth",
                    (rgbs_resized[ACQUISITION_T], xyzs[ACQUISITION_T, ..., 2]),
                ),
            ]
        )
    ),
    (jnp.arange(2), model_args, object_library),
)

In [ ]:
# Visualize trace
bayes3d.rerun_visualize_trace_t(trace, ACQUISITION_T)

In [ ]:
FINAL_T = len(xyzs)
for T_observed_image in tqdm(range(ACQUISITION_T, FINAL_T)):
    # Constrain on new RGB and Depth data.
    trace = b3d.update_choices(
        trace,
        Pytree.const(("observed_rgb_depth",)),
        (rgbs_resized[T_observed_image], xyzs[T_observed_image, ..., 2]),
    )
    # camera pose doesn't change in physion
    # trace, key = bayes3d.enumerate_and_select_best_move(
    #     trace, Pytree.const(("camera_pose",)), key, all_deltas
    # )
    trace, key = bayes3d.enumerate_and_select_best_move(
        trace, Pytree.const(("object_pose_1",)), key, all_deltas
    )
    bayes3d.rerun_visualize_trace_t(trace, T_observed_image)
